# Detecting stops

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas-examples/main?filepath=1-tutorials/8-detecting-stops.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/movingpandas/movingpandas-examples/blob/main/1-tutorials/8-detecting-stops.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://movingpandas.github.io/movingpandas-website/1-tutorials/8-detecting-stops.html)

There are no definitive answers when it comes to detecting / extracting stops from movement trajectories. Due to tracking inaccuracies, movement speed rarely goes to true zero. GPS tracks, for example, tend to keep moving around the object's stop location.

Suitable stop definitions are also highly application dependent. For example, an application may be interested in analyzing trip purposes. To do so, analysts would be interested in stops that are longer than, for example, 5 minutes and may try to infer the purpose of the stop from the stop location and time. Shorter stops, such as delays at traffic lights, however would not be relevant for this appication.

In the MovingPandas TrajectoryStopDetector implementation, a stop is detected if the movement stays within an area of specified size for at least the specified duration.

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas
import matplotlib.pyplot as plt
import folium

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts

import warnings

warnings.filterwarnings("ignore")

plot_defaults = {"linewidth": 5, "capstyle": "round", "figsize": (9, 3), "legend": True}
opts.defaults(
    opts.Overlay(active_tools=["wheel_zoom"], frame_width=500, frame_height=400)
)

mpd.show_versions()

In [ ]:
gdf = read_file("../data/geolife_small.gpkg")
tc = mpd.TrajectoryCollection(gdf, "trajectory_id", t="t")

## Stop detection with a single Trajectory

In [ ]:
my_traj = tc.trajectories[0]
my_traj

In [ ]:
traj_plot = my_traj.hvplot(
    title="Trajectory {}".format(my_traj.id),
    line_width=7.0,
    tiles="CartoLight",
    color="slategray",
)
traj_plot

In [ ]:
detector = mpd.TrajectoryStopDetector(my_traj)

### Stop duration

In [ ]:
%%time
stop_time_ranges = detector.get_stop_time_ranges(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
for x in stop_time_ranges:
    print(x)

### Stop points

In [ ]:
%%time
stop_points = detector.get_stop_points(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
stop_points

In [ ]:
stop_point_plot = traj_plot * stop_points.hvplot(
    geo=True, size="duration_s", color="deeppink"
)
stop_point_plot

In [ ]:
stop_points_gdf = gpd.GeoDataFrame(stop_points, geometry="geometry", crs="EPSG:4326")

In [ ]:
m = my_traj.explore(
    color="blue",
    style_kwds={"weight": 4},
    name="Trajectory",
)

stop_points_gdf.explore(
    m=m,
    color="red",
    style_kwds={
        "style_function": lambda x: {"radius": x["properties"]["duration_s"] / 10}
    },
    name="Stop points",
)

folium.TileLayer("OpenStreetMap").add_to(m)
folium.LayerControl().add_to(m)

m

### Stop segments

In [ ]:
%%time
stop_segments = detector.get_stop_segments(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
stop_segments

In [ ]:
stop_segment_plot = stop_point_plot * stop_segments.hvplot(
    line_width=7.0, tiles=None, color="orange"
)
stop_segment_plot

In [ ]:
m = my_traj.explore(
    color="blue",
    tooltip="trajectory_id",
    popup=True,
    style_kwds={"weight": 4},
    name="Trajectory",
)

stop_segments.explore(
    m=m,
    color="orange",
    tooltip="trajectory_id",
    popup=True,
    style_kwds={"weight": 4},
    name="Stop segments",
)

stop_points_gdf.explore(
    m=m,
    color="red",
    tooltip="stop_id",
    popup=True,
    marker_kwds={"radius": 3},
    name="Stop points",
)

folium.TileLayer("CartoDB positron").add_to(m)
folium.LayerControl().add_to(m)

m

### Split at stops

In [ ]:
%%time
split = mpd.StopSplitter(my_traj).split(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
split

In [ ]:
split.to_traj_gdf()

In [ ]:
split.explore(
    column="trajectory_id", tiles="CartoDB dark_matter", style_kwds={"weight": 4}
)

In [ ]:
stop_segment_plot + split.hvplot(
    title="Trajectory {} split at stops".format(my_traj.id),
    line_width=7.0,
    tiles="CartoLight",
)

## Stop Detection for TrajectoryCollections

The process is the same as for individual trajectories.

In [ ]:
%%time
detector = mpd.TrajectoryStopDetector(tc)
stop_points = detector.get_stop_points(
    min_duration=timedelta(seconds=120), max_diameter=100
)
len(stop_points)

In [ ]:
ax = tc.plot(figsize=(7, 7))
stop_points.plot(ax=ax, color="red")

In [ ]:
m = tc.explore(
    column="trajectory_id",
    tooltip="trajectory_id",
    popup=True,
    style_kwds={"weight": 4},
    name="Trajectories",
)

stop_points.explore(
    m=m,
    color="red",
    tooltip="stop_id",
    popup=True,
    marker_kwds={"radius": 5},
    name="Stop points",
)

folium.TileLayer("CartoDB positron").add_to(m)
folium.LayerControl().add_to(m)

m